In [1]:
import pandas as pd
# library for date manipulation
from datetime import timedelta

In [2]:
# data path
data_path ="btc-usd-max.csv"

In [4]:
events = {
    "China Regulations over Chinese Bitcoin Exchanges": {
        "startDate": "2017-01-02",
        "endDate": "2017-01-10"
    },
    "SEC Denial of BTC ETF": {
        "startDate": "2017-02-01",
        "endDate": "2017-03-25"
    },
    "Possible market manipulation by Tether + FOMO": {
        "startDate": "2017-06-01",
        "endDate": "2017-12-15"
    },
    "Introduction of Bitcoin Futures and Regulations": {
        "startDate": "2017-12-31",  
        "endDate": "2018-03-01"    
    },
    "Market Cap Falls Below 100 Billion Dollars": {
        "startDate": "2018-11-01",
        "endDate": "2018-12-01"
    },
    "Crypto Exchanges Launches and FOMO": {
        "startDate": "2019-04-01",
        "endDate": "2019-08-01"
    },
    "Covid Pandemic Impact on Crypto": {
        "startDate": "2020-03-01",
        "endDate": "2020-04-15"
    },
    "Stock Market Rally and Crypto Adoption Surge": {
        "startDate": "2020-10-01",
        "endDate": "2021-04-01"
    },
    "Tesla and China Affect on Crypto": {
        "startDate": "2021-05-01",
        "endDate": "2021-07-01"
    },
    "Elon musk talks with Leading Crypto companies for more sustainable cryptomining":{
        "startDate": "2021-07-01",
        "endDate": "2021-11-15"
    },
    "Market Correction":{
        "startDate": "2021-11-15",
        "endDate":"2022-01-20"
    },
    "Russia Invades Ukraine":{
        "startDate": "2022-02-24",
        "endDate": "2022-03-31"
    },
    "SEC Announces crypto regulation, crypto exchanges layoffs, interests rates rise, Coinbase stocks drop":{
        "startDate": "2022-04-01",
        "endDate":"2022-06-01"
    },
    "FTX currency crashes and FTX declares bankruptcy":{
        "startDate": "2022-11-04",
        "endDate": "2022-12-01"
    },
    "Shares in Silvergate Capital, SEC freezes BKCoin transactions and accuses fraud":{
        "startDate": "2023-03-01",
        "endDate": "2023-03-12"
    },
    "Israel attack on Gaza":{
        "startDate":"2023-10-07",
        "endDate": "2023-10-14"
    },
    "Bitcoin Spot ETF":{
        "startDate": "2023-10-23",
        "endDate": "2024-04-12"
    },
    "Iran Attack on Israel":{
        "startDate": "2024-04-13",
        "endDate": "2024-04-17"
    }
}

In [5]:
# read csv file
df = pd.read_csv(data_path, parse_dates=['snapped_at'])

df['snapped_at'] = df['snapped_at'].dt.tz_localize(None)  # Ensure timezone-naive

# Create a new column to track if a date falls within any event range
df['within_event'] = False

# Process each event
for event, details in events.items():
    start_date = pd.to_datetime(details["startDate"])

    # Determine the end date based on provided data
    if 'endDate' in details and details['endDate']:
        end_date = pd.to_datetime(details['endDate'])
        if 'duration' not in details or not details['duration']:  # Calculate duration if not provided
            details['duration'] = (end_date - start_date).days
    else:
        if 'duration' in details and details['duration']:
            duration = timedelta(days=details['duration'])
            end_date = start_date + duration
            details['endDate'] = end_date.strftime('%Y-%m-%d')  # Store computed end date in a consistent format
        else:
            raise ValueError("Either duration or endDate must be provided for each event.")

    # Mark rows within the event date range
    mask = (df['snapped_at'] >= start_date) & (df['snapped_at'] <= end_date)
    df.loc[mask, 'within_event'] = True

# Set values to "" for rows outside any event ranges
outside_event_mask = ~df['within_event']
df.loc[outside_event_mask, ['price', 'market_cap', 'total_volume']] = ""

# Clean up by removing the temporary 'within_event' column
df.drop('within_event', axis=1, inplace=True)

# Optionally, save the modified DataFrame back to a CSV
df.to_csv('removed_onEvent_btc-usd-max.csv', index=False)
